In [262]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import my_api_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000) & (df["PRIMGEO_FLAG"]==0)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [263]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
#     "Category 2": [],
#     "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for city, state in zip(random_rural_city["NAME"], random_rural_city["STNAME"]):
    for i in range(len(offset)):
        params = {
            'location': f"{city}, {state}",
            'radius': 8047,
            'limit': 50,
            'offset': offset[i]
                 }
        response = requests.get(url, params = params, headers = headers).json()
        try:
            for business in range(len(response["businesses"])):
                try:
                    rural_dict["Searched"].append(f"{city}, {state}")
                    rural_dict["City"].append(response["businesses"][business]["location"]["city"])
                    rural_dict["State"].append(response["businesses"][business]["location"]["state"])
                    rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
                    rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
                    rural_dict["Restaurant"].append(response["businesses"][business]["name"])
                    rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
#                     rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#                     rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#                     rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
                    rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
                except KeyError:
                    rural_dict["Searched"].append("No City Found")
                    rural_dict["City"].append("No City Found")
                    rural_dict["State"].append("No State Found")
                    rural_dict["Lat"].append("No Lat Found")
                    rural_dict["Lng"].append("No Lng Found")
                    rural_dict["Restaurant"].append("No Restaurant Found")
                    rural_dict["Category"].append("No Categories")
                    rural_dict["Transactions"].append("No Transactions Found")
#                 except IndexError:
#                     rural_dict["Category 2"].append("")
#                     rural_dict["Category 3"].append("")
        except KeyError:
                pass

0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900


In [264]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head(70)

,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions
0,"Ilion village, New York",Herkimer,NY,43.0254,-74.9865,Crazy Ottos Empire Diner,"[Coffee & Tea, Diners]",[delivery]
1,"Ilion village, New York",Ilion,NY,43.0172,-75.0338,Sorrento's Pizzeria & Restaurant,"[Pizza, Italian, Seafood]",[delivery]
2,"Ilion village, New York",Herkimer,NY,43.0231,-74.9933,Tokyo Restaurant,"[Sushi Bars, Japanese]",[delivery]
3,"Ilion village, New York",Herkimer,NY,43.0254,-74.981,PK's Pub,"[American (New), Pubs]",[]
4,"Ilion village, New York",Herkimer,NY,43.024,-74.9863,Jamo's Restaurant & Bar,"[Bars, American (Traditional)]",[delivery]
...,...,...,...,...,...,...,...,...
65,"Flemington borough, Pennsylvania",Lock Haven,PA,41.1374,-77.4447,Leo's Italian & Specialty Foods,"[Delis, Italian]",[]
66,"Flemington borough, Pennsylvania",Lock Haven,PA,41.1349,-77.4441,Dutch Haven Restaurant,"[Seafood, American (Traditional)]",[]
67,"Flemington borough, Pennsylvania",Lock Haven,PA,41.1353,-77.4511,Hangar 9 Restaurant & Lounge,"[Steakhouses, Sandwiches]",[]
68,"Flemington borough, Pennsylvania",Mill Hall,PA,41.1134,-77.4883,Creek Side Restaurant,[American (Traditional)],[]


In [223]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Category 2": [],
    "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city, state in zip(random_urban_city["NAME"], random_urban_city["STNAME"]):
    params = {
        'limit': 50,
        'offset': 50,
        'location': f"{city}, {state}",
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            urban_dict["Searched"].append(f"{city}, {state}")
            urban_dict["City"].append(response["businesses"][business]["location"]["city"])
            urban_dict["State"].append(response["businesses"][business]["location"]["state"])
            urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            urban_dict["Restaurant"].append(response["businesses"][business]["name"])
            urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            urban_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#             urban_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#             urban_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
            urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            urban_dict["Searched"].append("No City Found")
            urban_dict["City"].append("No City Found")
            urban_dict["State"].append("No State Found")
            urban_dict["Lat"].append("No Lat Found")
            urban_dict["Lng"].append("No Lng Found")
            urban_dict["Restaurant"].append("No Restaurant Found")
            urban_dict["Category"].append("No Categories")
            urban_dict["Transactions"].append("No Transactions Found")
        except IndexError:
            urban_dict["Category 2"].append("")
            urban_dict["Category 3"].append("")

NameError: name 'resopnse' is not defined

In [167]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()
urban.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Category 2,Category 3,Transactions
0,"Madison city, Alabama",Madison,AL,34.7011,-86.7418,Jack's,Burgers,,,[]
1,"Madison city, Alabama",Madison,AL,34.7524,-86.7464,Ricks Barbecue - Madison,Barbeque,Southern,,[]
2,"Madison city, Alabama",Madison,AL,34.7061,-86.7439,Old Mexico Cantina,Mexican,,,[]
3,"Madison city, Alabama",Madison,AL,34.7526,-86.7515,ChuckWagon BBQ,Barbeque,,,[]
4,"Madison city, Alabama",Madison,AL,34.7073,-86.7506,Culver's,Burgers,,Fast Food,[]


In [265]:
urban.to_csv("../urban.csv")
rural.to_csv("../rural.csv")

In [287]:
rural_cat= pd.DataFrame(rural["Category"].to_list(), columns = ["Category 1", "Category 2", "Category 3", "Category 4", "Category 5", "Category 6"])
rural_clean = rural_cat[["Category 1", "Category 2", "Category 3"]]
rural_clean.head()


,Category 1,Category 2,Category 3
0,Coffee & Tea,Diners,None
1,Pizza,Italian,Seafood
2,Sushi Bars,Japanese,None
3,American (New),Pubs,None
4,Bars,American (Traditional),None
